# Infer LYSOSOME - part 4

--------------

## OBJECTIVE: ✅ Infer sub-cellular component LYSOSOME  in order to understand interactome 



Dependencies:
The LYSOSOMES  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles.nuclei import infer_NUCLEI
from infer_subc_2d.organelles.soma import infer_SOMA
from infer_subc_2d.organelles.cytosol import infer_CYTOSOL

%load_ext autoreload
%autoreload 2

test_img_n = 5

---------------------

# IMAGE PROCESSING Objective 4:  infer LYSOSOME

## summary of steps (Workflow #1 & #2)

INPUT
- channel  2
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
- enhance two classes of "spots"
-  segment objects

POST-PROCESSING
  - filter objects

OUTPUT
- object LYSOSOME 



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [3]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [4]:
load_Z_from_params = False


if load_Z_from_params:

    default_params = load_parameters( test_img_name.split("/")[-1], data_root_path / "intermediate" )

    ch_to_agg = default_params["ch_to_agg"]
    nuc_ch = default_params['nuc_ch']
    optimal_Z = default_params["optimal_Z"] #find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
else:
    ch_to_agg = (1,2,3,4,5,6)
    nuc_ch = 0
    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    default_params = defaultdict(str, **{
        #"intensity_norm_param" : [0.5, 15]
        "intensity_norm_param" : [0],
        "gaussian_smoothing_sigma" : 1.34,
        "gaussian_smoothing_truncate_range" : 3.0,
        "dot_2d_sigma" : 2,
        "dot_2d_sigma_extra" : 1,
        "dot_2d_cutoff" : 0.025,
        "min_area" : 10,
        "low_level_min_size" :  100,
        "median_filter_size" : 4,
        "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
        "nuc_ch" : 0,
        "optimal_Z": optimal_Z,
    })
    save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


## get the inferred nuclei object

(takes < 1 sec)

In [5]:


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


## get the inferred soma object

(takes < 1 sec)

In [6]:
raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, default_params) 
CY_object =  infer_CYTOSOL(SO_object, NU_object) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [7]:
CY_object.dtype

dtype('bool')

# WORKFLOW #1 

Generally following the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.

> Using Allen Cell Segmenter LAMP1 [workflow](https://www.allencell.org/cell-observations/category/lamp1).  Examples sourced from: [Notebook](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/lookup_table_demo/playground_lamp1.ipynb) and [script](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_lamp1.py)

## summary of steps

INPUT
- channel 1
- CYTO mask

PRE-PROCESSING
- contrast scale
- ne-noise and somoothe

CORE PROCESSING
- dot enhancement / segment
- filiment enhancement / segment

POST-PROCESSING
  - fill holes
  - remove small objects


OUTPUT
- mask of LYSOSOME


### INPUT

In [8]:

###################
# INPUT
###################
raw_lyso    = img_2D[1].copy()

# mask object
#raw_lyso[~CY_object] = 0 
masked_lyso = apply_mask(raw_lyso,CY_object)


### PRE-PROCESSING

In [11]:
###################
# PRE_PROCESSING
###################

# intensity_norm_param = [0, 9] # from Allen Cell Segmenter LAMP1  workflow
intensity_norm_param = [0] 

# Linear-ish smoothing
lysosomes = intensity_normalization( masked_lyso ,  scaling_param=intensity_norm_param)

med_filter_size = 3   
structure_img_median = ndi.median_filter(lysosomes, size=med_filter_size)

gaussian_smoothing_sigma = 1.34
gaussian_smoothing_truncate_range = 3.0
structure_img_smooth = ndi.gaussian_filter( structure_img_median, 
                                                                            sigma=gaussian_smoothing_sigma, 
                                                                            mode="nearest", 
                                                                            truncate=gaussian_smoothing_truncate_range)
# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

intensity normalization: min-max normalization with NO absoluteintensity upper bound


### CORE PROCESSING

In [12]:
###################
# CORE_PROCESSING
###################
# dot and filiment enhancement - 2D

################################
## PARAMETERS for this step ##
s2_param = [[5,0.09], [2.5,0.07], [1,0.01]]
################################
bw_spot = dot_2d_slice_by_slice_wrapper(struct_img, s2_param)


################################
## PARAMETERS for this step ##
f2_param = [[1, 0.15]]
################################
bw_filament = filament_2d_wrapper(struct_img, f2_param)
bw = np.logical_or(bw_spot, bw_filament)



### POST-PROCESSING

> NOTE: we are using the 3D aicssegmentation functions which wrap `scipy.ndimage` functions with a single Z rather than the `scipy.ndimage` functions directly

In [13]:
###################
# POST_PROCESSING
###################

################################
## PARAMETERS for this step ##
fill_2d = True
fill_max_size = 25
hole_min = 1
################################

removed_holes = hole_filling(bw, hole_min ** 2, fill_max_size ** 2, fill_2d)


width = 3  
cleaned_img = size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**2, 
                                                         method = "slice_by_slice" ,
                                                         connectivity=1)


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [14]:

viewer = napari.view_image(
    removed_holes,
    scale=scale
)


In [15]:

viewer.scale_bar.visible = True
viewer.add_image(
    cleaned_img,
    scale=scale
)
viewer.add_image(
    structure_img_smooth,
    scale=scale
)

<Image layer 'structure_img_smooth' at 0x178ffbb50>

### DEFINE `infer_LYSOSOMES` function

In [20]:
##########################
#  infer_LYSOSOMES
##########################
def _infer_LYSOSOMES(struct_img,  CY_object, in_params) -> tuple:
    """
    Procedure to infer LYSOSOME from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the LYSOSOME signal

    CY_object: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of Lysosomes
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################     
    # 
    # 
    struct_img = apply_mask(struct_img,CY_object)
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

   # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 3   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # log_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization( log_img ,  scaling_param=[0] )  
    struct_img = intensity_normalization( struct_img ,  scaling_param=[0] )  


   ###################
    # CORE_PROCESSING
    ###################
    # dot and filiment enhancement - 2D
    ################################
    ## PARAMETERS for this step ##
    s2_param = [[5,0.09], [2.5,0.07], [1,0.01]]
    ################################
    bw_spot = dot_2d_slice_by_slice_wrapper(struct_img, s2_param)

    ################################
    ## PARAMETERS for this step ##
    f2_param = [[1, 0.15]]
    ################################
    bw_filament = filament_2d_wrapper(struct_img, f2_param)

    bw = np.logical_or(bw_spot, bw_filament)

    out_p["s2_param"] = s2_param 
    out_p["f2_param"] = f2_param 
    ###################
    # POST_PROCESSING
    ###################

    # 2D cleaning
    hole_min = 1
    hole_max = 25  
    # discount z direction
    struct_obj = hole_filling(bw, hole_min = hole_min **2 , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max


    # # 3D
    # cleaned_img = remove_small_objects(removed_holes>0, 
    #                                                             min_size=width, 
    #                                                             connectivity=1, 
    #                                                             in_place=False)

    small_object_max = 3
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**2, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

                
    retval = (struct_obj,  out_p)
    return retval


------------------

# TEST `infer_LYSOSOMES` function

In [21]:
###################
# SOMA INPUT
###################

img_2D = img_data[:,[optimal_Z],:,:].copy()
raw_lyso =  img_2D[1].copy()


LY_object, out_p =  _infer_LYSOSOMES(raw_lyso, CY_object, default_params) 

intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [22]:

viewer.add_image(
    LY_object,
    scale=scale
)
viewer.add_labels(
    label(LY_object),
    scale=scale
)

<Labels layer 'Labels' at 0x181e91ca0>

In [23]:
from infer_subc_2d.organelles.lysosomes import infer_LYSOSOMES


img_2D = img_data[:,[optimal_Z],:,:].copy()
raw_lyso =  img_2D[1].copy()


LY_object, out_p =  infer_LYSOSOMES(raw_lyso, CY_object, default_params) 



intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [24]:

viewer.add_image(
    LY_object,
    scale=scale
)

<Image layer 'LY_object [1]' at 0x182501430>